In [1]:
import numpy as np
import re
import pandas as pd

In [2]:
import re
import pandas as pd

# ログファイルの読み込み
params = {}
with open('fit.log', 'r') as f:
    lines = f.readlines()

    # 下側から「Final set of parameters」を探す
    start_index = None
    for i in range(len(lines)-1, -1, -1):  # 下から上に向かって探索
        if "Final set of parameters" in lines[i]:
            start_index = i
            break

    # 「Final set of parameters」から次のセクションまでのデータを取得
    if start_index is not None:
        for line in lines[start_index + 2:]:  # 最初の2行はヘッダー行なので無視
            if "correlation matrix" in line:  # 相関行列が始まったら終了
                break

            # 正規表現で「パラメータ名 = 値 +/- 誤差」を抽出
            match = re.match(r'(\w+)\s*=\s*([\d\.e+-]+)\s*\+/-\s*([\d\.e+-]+)', line)
            if match:
                param, value, error = match.groups()
                # 数値変換
                try:
                    params[param] = (float(value), float(error))
                except ValueError:
                    print(f"エラー: {line.strip()} は正しい数値形式ではありません。")

# 結果を表示
if params:
    for param, (value, error) in params.items():
        print(f"{param} = {value:.4f} ± {error:.4f}")

    # データフレームに変換してCSVに保存
    df = pd.DataFrame.from_dict(params, orient='index', columns=['Value', 'Error'])
    df.to_csv('fit_parameters_1480V.csv', index_label='Parameter')

    print("フィッティングパラメータと誤差を 'fit_parameters_1480V.csv' に保存しました。")
else:
    print("データの抽出に失敗しました。ログファイルの内容を確認してください。")

a11 = 312.8010 ± 2.6100
a12 = 25.2981 ± 0.4982
a13 = 69.2913 ± 0.4615
a21 = 62.2157 ± 1.4490
a22 = 7.8271 ± 1.3260
a23 = 52.7936 ± 1.5960
フィッティングパラメータと誤差を 'fit_parameters_1480V.csv' に保存しました。


In [3]:
#フィッティングように足した定数高を足して調整
#複数回実行するとその分加算されるので注意

import pandas as pd

# CSVファイルの読み込み
df = pd.read_csv('fit_parameters_1480V.csv')

# 'Value'列の取り出し
values = df['Value'].values
# 'Error'列の取り出し
errors = df['Error'].values

# 数値データを個別に取り出す
a11_value = df.loc[df['Parameter'] == 'a11', 'Value'].values[0]
a11_error = df.loc[df['Parameter'] == 'a11', 'Error'].values[0]
a12_value = df.loc[df['Parameter'] == 'a12', 'Value'].values[0]
a22_value = df.loc[df['Parameter'] == 'a22', 'Value'].values[0]

# a12に1200を加算
a12_value = a12_value + 1100
a22_value = a22_value + 870

# DataFrameの該当する行を更
df.loc[df['Parameter'] == 'a12', 'Value'] = a12_value
df.loc[df['Parameter'] == 'a22', 'Value'] = a22_value

# 変更をCSVファイルに上書き保存
df.to_csv('fit_parameters_1480V.csv', index=False)

print("CSVファイルに上書き保存しました。")







CSVファイルに上書き保存しました。


In [4]:

# CSVファイルの読み込み
df = pd.read_csv('fit_parameters_1480V.csv')

# 'Value'列と'Error'列の数値データを取り出す
values = df['Value'].values
errors = df['Error'].values

a11_value = df.loc[df['Parameter'] == 'a11', 'Value'].values[0]
a11_error = df.loc[df['Parameter'] == 'a11', 'Error'].values[0]
a12_value = df.loc[df['Parameter'] == 'a12', 'Value'].values[0]
a12_error = df.loc[df['Parameter'] == 'a12', 'Error'].values[0]
a13_value = df.loc[df['Parameter'] == 'a13', 'Value'].values[0]
a13_error = df.loc[df['Parameter'] == 'a13', 'Error'].values[0]
a21_value = df.loc[df['Parameter'] == 'a21', 'Value'].values[0]
a21_error = df.loc[df['Parameter'] == 'a21', 'Error'].values[0]
a22_value = df.loc[df['Parameter'] == 'a22', 'Value'].values[0]
a22_error = df.loc[df['Parameter'] == 'a22', 'Error'].values[0]
a23_value = df.loc[df['Parameter'] == 'a23', 'Value'].values[0]
a23_error = df.loc[df['Parameter'] == 'a23', 'Error'].values[0]

# henkanとhenkanmissを計算
henkan = 5.9 / a12_value
henkan_error = 5.9 * a12_error / a12_value**2

# 必要な値の計算
FWHM_main=2*np.sqrt(2*np.log(2))*a13_value
FWHM_main_keV=FWHM_main*henkan
FWHM_main_error=2*np.sqrt(2*np.log(2))*a13_error
FWHM_main_error_keV=np.sqrt(FWHM_main**2 *  henkan_error**2+ henkan**2 *FWHM_main_error**2)
energy_resolution_main=FWHM_main_keV*100/5.9
energy_resolution_error_main=FWHM_main_error_keV*100/5.9

peak_value_main=a11_value
peak_value_main_error=a11_error

sub_peak=a22_value*henkan
sub_peak_error=np.sqrt(sub_peak**2*henkan_error**2+henkan**2*a22_error**2)

FWHM_sub=2*np.sqrt(2*np.log(2))*a23_value
FWHM_sub_keV=FWHM_sub*henkan
FWHM_sub_error=2*np.sqrt(2*np.log(2))*a23_error
FWHM_sub_error_keV=np.sqrt(FWHM_sub**2*henkan_error**2+henkan**2*FWHM_sub_error**2)
energy_resolution_sub=FWHM_sub_keV*100/5.9
energy_resolution_error_sub=np.sqrt(FWHM_sub_error_keV**2/sub_peak**2+FWHM_sub_keV**2*sub_peak_error**2/sub_peak**4)

peak_value_sub=a21_value
peak_value_sub_error=a21_error

print(henkan,'±',henkan_error)

0.005243055151341676 ± 2.321242767937156e-06


In [5]:
print('主ピークのピーク値:', peak_value_main,'±',peak_value_main_error)
print('主ピークのFWHM(keV):', FWHM_main_keV,'±',FWHM_main_error_keV)
print('主ピークのエネルギー分解能:', energy_resolution_main,'±',energy_resolution_error_main)
print('サブピークのピーク値:', peak_value_sub,'±',peak_value_sub_error)
print('サブピークの中心(keV):', sub_peak,'±',sub_peak_error)
print('サブピークのFWHM(keV):', FWHM_sub_keV,'±',FWHM_sub_error_keV)
print('サブピークのエネルギー分解能:', energy_resolution_sub,'±',energy_resolution_error_sub)



主ピークのピーク値: 312.801 ± 2.61
主ピークのFWHM(keV): 0.8555016656465456 ± 0.005710461819620509
主ピークのエネルギー分解能: 14.500028231297382 ± 0.0967874884681442
サブピークのピーク値: 62.2157 ± 1.449
サブピークの中心(keV): 4.602495793781221 ± 0.006952299339291017
サブピークのFWHM(keV): 0.6518136149195853 ± 0.019707049340414094
サブピークのエネルギー分解能: 11.047688388467549 ± 0.004287158738528886


In [6]:
with open('fit_results_1480.txt', 'w') as file:
    file.write(f"{peak_value_main} {peak_value_main_error}\n")
    file.write(f"{FWHM_main_keV} {FWHM_main_error_keV}\n")
    file.write(f"{energy_resolution_main} {energy_resolution_error_main}\n")
    file.write(f"{peak_value_sub} {peak_value_sub_error}\n")
    file.write(f"{sub_peak} {sub_peak_error}\n")
    file.write(f"{FWHM_sub_keV} {FWHM_sub_error_keV}\n")
    file.write(f"{energy_resolution_sub} {energy_resolution_error_sub}\n")

print("数値データが 'fit_results_1480.txt' に保存されました。")

数値データが 'fit_results_1480.txt' に保存されました。
